In [46]:
# TEST GPU
import torch


print('Cuda availability :',torch.cuda.is_available())
print('How many cuda device :', torch.cuda.device_count())
cuda_device = torch.cuda.current_device()
print('Name of current device :',torch.cuda.get_device_name(cuda_device))

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

device = get_default_device()

Cuda availability : True
How many cuda device : 1
Name of current device : NVIDIA GeForce RTX 3090


In [45]:
# pass data to device
def to_device(data, device):
    """Move tensors to chosen device"""
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [49]:
class DeviceDataLoader:
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield  to_device(b, self.device)

    def __len__(self):
        """number of batches"""
        return len(self.dl)

In [3]:
import torch
import numpy as np
import torchvision
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader

In [4]:
dataset = MNIST(root='data/',
                download=True,
                transform=ToTensor())

/home/divclab/anaconda3/envs/torch-env/lib/python3.6/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448272031/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
def split_indices(n, val_percent):

    # size of validation set
    n_val = int(val_percent * n)
    # random permutation of 0 to n-1
    idxs = np.random.permutation(n)

    return idxs[n_val:], idxs[:n_val]

In [10]:
train_indices, val_indices = split_indices(len(dataset), val_percent=0.2)

print(len(train_indices), len(val_indices))


48000 12000


In [54]:
batch_size = 100

train_sampler = SubsetRandomSampler(train_indices)
train_dl = DataLoader(dataset,
                      batch_size,
                      sampler=train_sampler)
train_dl = DeviceDataLoader(train_dl, device)

valid_sampler = SubsetRandomSampler(val_indices)
valid_dl = DataLoader(dataset,
                      batch_size,
                      sampler=valid_sampler)
valid_dl = DeviceDataLoader(valid_dl, device)

In [56]:
for xb, yb in valid_dl:
    print('xb.device:', xb.device)
    print('yb:', yb)

    break

xb.device: cuda:0
yb: tensor([9, 4, 9, 7, 7, 8, 0, 0, 8, 3, 7, 6, 3, 6, 3, 7, 9, 5, 8, 7, 7, 5, 1, 3,
        9, 1, 5, 8, 5, 5, 2, 8, 4, 3, 3, 2, 7, 0, 3, 1, 5, 0, 6, 4, 9, 2, 8, 1,
        3, 8, 5, 7, 3, 6, 7, 6, 4, 4, 2, 1, 6, 2, 0, 2, 1, 2, 3, 9, 5, 4, 1, 5,
        0, 8, 2, 7, 0, 3, 5, 6, 5, 8, 7, 6, 2, 7, 6, 5, 9, 4, 9, 9, 4, 5, 8, 8,
        4, 7, 3, 3], device='cuda:0')


In [12]:
import torch.nn.functional as F
import torch.nn as nn

In [21]:
for xb, yb in train_dl:
    print(xb.shape)
    xb = xb.view(xb.size(0),784)
    print(xb.shape)
    break

torch.Size([100, 1, 28, 28])
torch.Size([100, 784])


In [36]:
class MnistModel(nn.Module):
    """feedforward neural network with 1 hidden layer"""
    def __init__(self, in_size, hidden_size, out_size):
        super().__init__()

        # hidden layer
        self.linear1 = nn.Linear(in_size, hidden_size)

        # output layer
        self.linear2 = nn.Linear(hidden_size, out_size)

    def forward(self, xb):

        # flatten (-1 means auto calculate)
        xb = xb.view(xb.size(0), -1)

        out = self.linear1(xb)
        out = F.relu(out)

        out = self.linear2(out)

        return out

In [37]:
input_size = 28*28
num_classes = 10

model = MnistModel(input_size, hidden_size=32, out_size=num_classes)

# chekcout parameter shapes
for t in model.parameters():
    print(t.shape)


torch.Size([32, 784])
torch.Size([32])
torch.Size([10, 32])
torch.Size([10])


In [42]:
for images, labels in train_dl:
    print('images.shape', images.shape)
    outputs = model(images)
    loss = F.cross_entropy(outputs, labels)
    print('Loss:', loss.item())
    break

print('outputs.shape:', outputs.shape)
print('sample outputs:\n', outputs[0].data)
print('after softmax:\n', F.softmax(outputs, dim=0)[0].data)





images.shape torch.Size([100, 1, 28, 28])
Loss: 2.309837579727173
outputs.shape: torch.Size([100, 10])
sample outputs:
 tensor([-0.1396,  0.1127,  0.2892,  0.0438,  0.0515, -0.0080,  0.0582, -0.0077,
        -0.2485, -0.1919])
after softmax:
 tensor([0.0101, 0.0098, 0.0110, 0.0102, 0.0099, 0.0097, 0.0098, 0.0109, 0.0083,
        0.0084])
